In [160]:
#!cat bibliotecas.txt | xargs -n 1 pip3 install

In [161]:
#Importação de bibliotecas
import RPi.GPIO as GPIO
import time, copy
import board, busio, adafruit_vl53l0x
import serial
import numpy as np
from numpy import save
from datetime import datetime


In [162]:
#Setup das portas logicas do Raspibery PI
class SetupGPIO:
    def __init__(self):
        self.objGPIO = GPIO
        self.objGPIO.cleanup() # limpa todos os estados de todas as portas
        self.objGPIO.setmode(GPIO.BCM) #Definindi uso dos numeros das portas por canais
    
    def get_gpio(self):
        return self.objGPIO

In [163]:
class MotorCarro:
    def __init__(self, raspGPIO):
        #GPIO Rodas dianteiras
        self.RDEF = 20
        self.RDDF = 5
        self.RDER = 21
        self.RDDR = 27

        #GPIO Rodas dianteiras
        self.RTEF = 12
        self.RTDF = 22
        self.RTER = 16
        self.RTDR = 17
        self.raspGPIO = raspGPIO
        self.raspGPIO.setup(self.RDEF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RDDF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RDER, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RDDR, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTEF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTDF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTER, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTDR, self.raspGPIO.OUT)
        self.stop()

    def stop(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDDF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDER, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDDR, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTER, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTDR, self.raspGPIO.LOW)

    def forward(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RDDF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.HIGH)
    

    def backward(self):
        self.raspGPIO.output(self.RDER, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RDDR, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTER, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDR, self.raspGPIO.HIGH)
        
    def left_forward(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDDF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.HIGH)



    def right_forward(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RDDF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.HIGH)

    def left_backward(self):
        self.raspGPIO.output(self.RDER, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDDR, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDR, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTER, self.raspGPIO.HIGH)

    def right_backward(self):
        self.raspGPIO.output(self.RDER, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RDDR, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTDR, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTER, self.raspGPIO.HIGH)
    
    def movimentacarro(self, movimento):
        if movimento==0:
            self.forward()
            time.sleep(1)
        elif movimento==1:
             self.backward()
             time.sleep(1)
        elif movimento==2:
             self.left_forward()
             time.sleep(2)
        elif movimento==3:
             self.right_forward()
             time.sleep(2)
        elif movimento==4:
             self.left_backward()
             time.sleep(1)
        elif movimento==5:
             self.right_backward()
             time.sleep(1)

        self.stop()


In [164]:
class Gsm_GPS:
    def __init__(self):
        self.port=serial.Serial("/dev/tty8",baudrate=11500, timeout=5  )
        self.port.flush()
        self.habilita_gsm_gps()
    
    def habilita_gsm_gps(self):
        self.port.write('AT\r'.encode())
        self.response()
        self.port.write('AT+CSQ\r'.encode())
        self.response()
        port.write('AT+CPIN?\r')
        response()
    
    def response(self):
        responseCount = 0
        while True:
            resp = self.port.readline().decode()
            print(resp)
            print(responseCount)
            if "OK" in resp:
                break
            if "ERROR" in resp:
                break
            if responseCount > 30:
                break
            else:
                responseCount +=1

In [165]:
#gsmgps = Gsm_GPS()

In [166]:
#movimentação do servo motor

class StepMotor:
    def __init__(self, raspGPIO, radar):
        self.controlPinEsquerda = [26,19,13,6]
        self.controlPinDireita = [6,13,19,26]
        self.posicoesMedicaRadar = [0,50, 100, 150, 200, 250, 299]
        self.medicaoRadar=[]
        self.raspGPIO = raspGPIO
        self.posicao_final = 0 #Esquerda
        
        self.seq1 = [[1,0,0,0], 
               [1,1,0,0],
               [0,1,0,0],
               [0,1,1,0],
               [0,0,1,0],
               [0,0,1,1],
               [0,0,0,1],
               [1,0,0,1]]
        self.seq2 = [
                [1,0,0,1],
                [1,0,0,0], 
                [1,1,0,0],
                [0,1,0,0],
                [0,1,1,0],
                [0,0,1,0],
                [0,0,1,1],
                [0,0,0,1]]
        self.parar_motor()

    def parar_motor(self):
        for pin in self.controlPinEsquerda:
            raspGPIO.setup(pin, raspGPIO.OUT)
            raspGPIO.output(pin, False)    
    
    def moverMotorEsquerda(self):
        self.medicaoRadar=[]
        for i in range(0, 300,1):
            if i in self.posicoesMedicaRadar:
                self.medicaoRadar.append(radar.get_radar_posicao())
                        
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinEsquerda[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)

    def moverMotorDireita(self):
        self.medicaoRadar=[]
        for i in range(0,300,1):
            if i in self.posicoesMedicaRadar:
                self.medicaoRadar.append(radar.get_radar_posicao())
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinDireita[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
    

    def moverMotorDireita_ajuste(self, valor):
        for i in range(0,valor,1):
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinDireita[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
       
    def moverMotorEsquerda_ajuste(self, valor):
        for i in range(0, valor, 1):
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinEsquerda[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
        
    def get_posicoes_radar(self):
        if self.posicao_final == 0:
            self.moverMotorEsquerda()
            self.posicao_final = 1
        else:
            self.moverMotorDireita()
            self.posicao_final = 0
            self.medicaoRadar = self.medicaoRadar[::-1] 
            
        return self.medicaoRadar

In [167]:
class Vl53_Radar:
    def __init__(self):
        self.i2c = busio.I2C(board.SCL, board.SDA)
        self.vl53 = adafruit_vl53l0x.VL53L0X(self.i2c)
        self.yarr = []
        self.count = 0

    
    def get_radar(self):
        return self.vl53
    
    def get_radar_posicao(self):
        return int(self.vl53.range / 10)

In [168]:
def mkdir(base):
    ''''
    Cria diretórios para salvar as matrizes 
    '''
    path = os.path.join('save', base)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def salvaMatrizes(matrizNome, matrizValor):
    ''''
    Cria diretórios para salvar as matrizes 
    Salva as matrizes com o nome e o valor passado como parametro
    '''
    today=datetime.today().strftime('%Y-%m-%d')
    mkdir(today)
    daytime=datetime.today().strftime('%H:%M:%S')
    np.savez(('save/%s/%s_%s.npz' % (today, matrizNome, daytime)), matrizValor)

def carregaMatriz(pasta, matrizNome):
    ''''
    Carrega as matrizes salvas no diretório
    '''
    path = os.path.join('save', pasta, matrizNome)
    load = np.load(path)
    return load['arr_0']

In [169]:
class CarEnv:
    def __init__(self, motorCarro, radarPosicoes, StepMotor):
        self.actions_space = ['forward', 'backward', 'leftforward', 'rightforward', 'leftbackward', 'rightbackward']
        self.observation_space = np.array(np.zeros([len(radarPosicoes), len(self.actions_space)]))
        self.state = radarPosicoes
        self.done = False
        self.input_size = len(radarPosicoes)
        self.output_size = len(self.actions_space)
        self.motorCar =  motorCarro
        self.finishCount = 0
    
    def finish(self, state):
        '''
        Quando o carro se movimentar 3 vezes para a frente sem parar é o objetivo dele
        '''
        print("STATE FINISH:", state)
        if(state[2]>100 and state[3]>100 and state[4]>100):
            self.finishCount+=1
        else:
            self.finishCount=0
        
        print("Count FINISH:", self.finishCount)

        self.finishStatus = False
        if self.finishCount>=3:
            self.finishStatus = True
        return self.finishStatus

    def step(self, action):
        self.take_action(action)
        self.state = self.getState()
        self.done = self.finish(self.state)
        stepP = copy.deepcopy(self.getReward()), self.state, self.done
        return stepP

    def getState(self):
        try:
            self.state = stepMotor.get_posicoes_radar()
        finally:
           stepMotor.parar_motor()
        return self.state
        
    def take_action(self, action):
        movPosition = np.where(action == np.max(action)) 
        l = list(action).index(np.max(action))
        print('action', l, np.max(action), action)
        self.motorCar.movimentacarro(l)

    def getReward(self):
        f = lambda x: 10 if  x>100 else -10
        return f(max(self.state[2:5]))


### Inicialização dos Hiperparametros
Neste parte implementaremos a seguinte parte do código

<img src="imagens/ars_part1.png" width=600 heigth=400>

Inicialmente vamos usar steps_size=10 e epsodes=10 apenas para fins de comparação de resultados, ja que o ambiente é real de um carro autonomo

self.directions = Total de matrizes de pertubações a serem contruidas ***number of directions sampled per iteration N***

self.best_directions = Total de matrizes com recompensas melhores 

Nunca as matrizes de pertubação pode ser maior que as matrizes de recompensas, por isso o uso do assert

***(number of top-performing directions to use b (b < N is allowed only for V1-t and V2-t)***

self.noise = noise ν

In [170]:
class Hiperparametros():
    def __init__(self):
        self.steps = 10
        self.epsodes = 10
        self.lr = 0.02
        self.directions = 6
        self.best_directions = 6
        assert self.best_directions <= self.directions
        self.noise = 0.03
        self.seed = 1
        self.env_nome = ''


Os bons resultados desse algoritmo se dão fortemente por causa da normalização dos dados

De acordo com documento, a normalização é necessário por:

"A normalização de estados usada por V2 é semelhante ao clareamento de dados usado em tarefas de regressão, e
intuitivamente, garante que as políticas atribuam peso igual aos diferentes componentes dos estados. Para
obter intuição de por que isso pode ajudar, suponha que uma coordenada de estado só tenha valores no intervalo
90, 100 enquanto outro componente de estado assume valores na faixa -1, 1. Então, pequenas mudanças em
o ganho de controle em relação à primeira coordenada de estado levaria a mudanças maiores nas ações
então, o mesmo tamanho muda em relação ao segundo componente de estado. Portanto, o clareamento permite
a exploração isotrópica de pesquisa aleatória para ter igual influência sobre os vários componentes de estado"


Nesse caso vamos normalizar os valores recebidos pelo radar que varia entre 1 e 899, que seriam equivalentes há 1cm e 89,9cm. 


In [171]:
#normalizacao dos estados (Standardization)
class Normalizacao():
    def __init__(self, inputs):
        '''
        Inicializa todos os parametros utilizados durante a normalização
        Keyword arguments:
        inputs -- array dos valores recebidos do sensor de distancia, ex: [30, 6, 3, 4, 8, 91, 819]
        '''
        self.n = np.zeros(inputs) #agregador de estados descobertos desde o inicio
        self.mean = np.zeros(inputs) #média de todos os valores de input
        self.mean_diff = np.zeros(inputs) #usado para o calculo da variancia
        self.var = np.zeros(inputs) #guarda os valores de variancia
    
    def observe(self, inputs):
        '''
        Realiza o calculo da variancia nos dados recebidos do sensor de movimento
        Keyword arguments:
        inputs -- array dos valores recebidos do sensor de distancia, ex: [30, 6, 3, 4, 8, 91, 819]
        '''
        self.n +=1. #indica em que ação é a atual
        last_mean = self.mean.copy() #guarda o valor da ultima média realizada 
        self.mean += (inputs - self.mean) / self.n #atualizando a média baseada na quantidade de ações ja realizadas
        self.mean_diff += (inputs - last_mean) * (inputs - self.mean) # pega a diferença atual e a média antiga
        self.var = (self.mean_diff/self.n).clip(min = 1e-2) #realiza o calculo da variancia, e limita o valor minino em 0.01

    def normalize(self, inputs):
        ''''
        Realiza o calculo da normalização(Padronização) dividindo realizando x- média(x) / desvio padrão de X
        Assim deixando todos os valores dentro da escala -1 e 1
        Essa forma é mais robusta contra outliers

        Keyword arguments:
        inputs -- array dos valores recebidos do sensor de distancia, ex: [30, 6, 3, 4, 8, 91, 819]

        Return:
        Normalized - Valores normalizados entre -1 e 1
        '''
        obs_mean = self.mean #Qual a média atual
        obs_std = np.sqrt(self.var) #Calcula Desvio padrão
        normalized = (inputs - obs_mean) / obs_std #(valor a ser nomalizado - média) / desvio padrão 
        return normalized

Neste parte implementaremos as seguintes partes do código

<img src="imagens/ars_part2.png" width=600 heigth=400>

<img src="imagens/ars_part3.png" width=600 heigth=400>


In [172]:
class Politicas():
    '''
        A exploração acontece no espaço das politicas depois que todo um episódio e steps dele foram executados
        Diferente de outras IAs que usam exploração por ambientes e ações executadas
    '''

    def __init__(self, input_size, output_size):
        '''
        Keyword arguments:
        input_size -- Numeros de entradas
        output_size -- Numeros de saídas

        Return:
        '''
        self.theta = np.zeros((output_size, input_size)) #Cria uma matrix de pesos inicializado em zeros
        #Estamos seguindo o método do artivo pelo lado esquerdo, por isso usamos (output_size, input_size)
    
    def evaluate(self, input, delta=None, direction=None):
        ''''
        De acordo com a direção passada em direction é atualizada a matriz de pesos

        Keyword arguments:
        inputs -- array dos valores recebidos do sensor de distancia, ex: [30, 6, 3, 4, 8, 91, 819]
        delta -- Matrix de pertubação dos numeros 
        direction -- Indica a direção do calculo para positivo ou negativo
        '''

        if direction is None:
            return self.theta.dot(input) #retorna a matriz de pesos que multiplica com as entradas, sem pertubações
        elif direction == 'positive':
            return (self.theta + hp.noise * delta).dot(input) #retorna a matriz de pesos que multiplica com as entradas, com ruido de exploração mais pertubações positivas
        else: 
            return (self.theta - hp.noise * delta).dot(input)#retorna a matriz de pesos que multiplica com as entradas, com ruido de exploração mais pertubações negativas
    
    def samples_deltas(self):
        '''
        Gerando matrix de pertubação, matrix com numeros aleatórios
        '''
        return [np.random.randn(*self.theta.shape) for _ in range(hp.directions)]
        #retorna uma matrix com distribuição normal para o todas as matrizes de pertubaçao 
    
    def update(self, rollouts, sigma_r):
        '''
        Item 7 do Algoritmo, fazendo a atualização dos pesos

        Keyword arguments:
        rollouts -- conjunto de recompensa positiva, conjunto recompensa negativa e a matrizx de numeros aleatórios
        sigma_r -- Indica o desvio padrão da recompensa
        '''
        step = np.zeros(self.theta.shape) #Inicializa com as dimensões de pesos
        for r_pos, r_neg, d in rollouts:
            step += (r_pos - r_neg) * d #Somatoria das recompensas positivas e negativas e a multicao do delta
        self.theta += hp.learning_rate / (hp.best_directions * sigma_r) * step #Atualizando a matriz  de pesos

In [173]:
def explore(env, normalizer, policy, direction = None, delta=None):
    '''
         Faz a exploração do ambiente enquanto não finalizar e não terminar as execuções do epsodio
      
    '''
    state = env.getState() #ler o ambiente radar
    done = False #inicia em False o objetivo
    num_plays = 0. #contador de rodadas no episódio
    sum_rewards = 0 #soma das recompensas
    while not done and num_plays < hp.epsodes:
       print('Execucao: ', num_plays, ', Episodio: ', hp.epsodes, ' Finalizado: ', done)
       normalizer.observe(state) #Atualiza o calculo da variancia nos dados recebidos do sensor de movimento
       state = normalizer.normalize(state) #Realiza o calculo da normalização(Padronização) deixando todos os estados entre -1 e 1
       action = policy.evaluate(state, delta, direction) #atualizada a matriz de pesos de acordo com a direçao selecionada e retirna 
       reward, state, done = env.step(action) #Executa a ação selecionada e retirna a nova leitura do ambiente e se foi finalizado 
       print("Recompensa: ", reward)
       reward = max(min(reward, 1), -1)#evita outlier nas recompensas
       sum_rewards += reward #Soma das recompensas
       num_plays +=1 #atualida o numero da rodada
    return sum_rewards


Neste parte implementaremos as seguintes partes do código

<img src="imagens/ars_part4.png" width=600 heigth=400>

In [174]:
#Treinando
def train(env, policy, normalizer, hp):
    ''''
      Realiza o treinamento da rede
   
    '''
    loadMatrixPositiveFilename = None
    LoadMatrixFolder = None
    loadMatrixNegativeFilename = None
    DeltaFilename = None
    for step in range(hp.steps):
        
        if loadMatrixPositiveFilename and loadMatrixNegativeFilename and LoadMatrixFolder and DeltaFilename:
            deltas = carregaMatriz(LoadMatrixFolder, loadMatrixPositiveFilename) #Inicializacao das pertubacoes (deltas) e as recompensas negativas e positivas)
            #positive_rewards = 
            negative_rewards = carregaMatriz(LoadMatrixFolder, DeltaFilename)
        else:
            deltas = policy.samples_deltas() #Inicializacao das pertubacoes (deltas) e as recompensas negativas e positivas)
            positive_rewards = [0] * hp.directions #inicializando a matriz de recompensas positivas
            negative_rewards = [0] * hp.directions #inicializando a matriz de recompensas negativas

        #obtendo as recompensas na direcao positiva
        for k in range(hp.directions):
            positive_rewards[k] = explore(env, normalizer, policy, direction='positive', delta=deltas[k])
            
        #obtendo recompensa na direcao negativa
        for k in range(hp.directions):
            positive_rewards[k] = explore(env, normalizer, policy,  direction='negative', delta=deltas[k])
        
        #obtendo todas as recompensas positivas e negativas para computar o desvio dessas recompensas
        all_reward = np.array(positive_rewards + negative_rewards)
        sigma_r = all_rewards.std()

        #ordenacao dos rollouts e selecao das melhores direcoes
        scores = {k: max(r_pos, r_neg) for k, (r_pos, r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
        order = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:hp.best_directions]
        rollouts = [(positive_rewards[k], negative_rewards[k], deltas [k]) for k in order]

        #atualizacao de politica
        policy.update(rollouts, sigma_r)

        #impressao da recompensa
        reward_evaluation = explore(env, normalizer, policy)
        print('Step', step, 'Reward:',reward_evaluation )

    salvaMatrizes('positive_rewards', positive_rewards)
    salvaMatrizes('negative_rewards', negative_rewards)
    alvaMatrizes('deltas', deltas)

    
             

In [175]:

raspGPIO  = SetupGPIO().get_gpio()
radar     = Vl53_Radar()
stepMotor = StepMotor(raspGPIO, radar)
motorCar = MotorCarro(raspGPIO)
carEnv = CarEnv(motorCar, stepMotor.get_posicoes_radar(),stepMotor)


In [176]:
 hp = Hiperparametros()
 np.random.seed(hp.seed)
 police = Politicas(carEnv.input_size, carEnv.output_size)
 normalizer = Normalizacao(carEnv.input_size)
 try:
    train(carEnv,police,normalizer,hp)
    pass
 finally:
     stepMotor.parar_motor()
 #Fim

Execucao:  0.0 , Episodio:  10  Finalizado:  False
action 0 0.0 [0. 0. 0. 0. 0. 0.]
Recompensa:  10
Execucao:  1.0 , Episodio:  10  Finalizado:  False
action 0 0.10974709017905396 [ 0.10974709 -0.07301745  0.08381035  0.01854194  0.00093107  0.08778463]
Recompensa:  10
Execucao:  2.0 , Episodio:  10  Finalizado:  False
action 4 0.003023498059221339 [-0.09707547 -0.05484158 -0.03575612 -0.03677758  0.0030235  -0.02669773]
Recompensa:  10
Execucao:  3.0 , Episodio:  10  Finalizado:  False
action 5 0.05512744928456858 [-0.10314292 -0.06644181  0.03960878 -0.03785251 -0.05893177  0.05512745]
Recompensa:  10
Execucao:  4.0 , Episodio:  10  Finalizado:  False
action 1 0.07890132557251969 [-0.00900117  0.07890133  0.01415069  0.03451897  0.06220336  0.04410733]
Recompensa:  10
Execucao:  5.0 , Episodio:  10  Finalizado:  False
action 4 0.10026044205389141 [0.03371261 0.08663423 0.01415866 0.00204447 0.10026044 0.01838037]
Recompensa:  10
Execucao:  6.0 , Episodio:  10  Finalizado:  False
acti

KeyboardInterrupt: 

In [159]:
stepMotor.moverMotorEsquerda_ajuste(300)
##stepMotor.get_posicoes_radar()
stepMotor.parar_motor()



In [177]:
motorCar.stop()

In [154]:
motorCar.movimentacarro(5)